In [2]:
import numpy as np
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.7.0 & Gurobi 9.0

### Problem Description
In this problem, we have six end distributors, each with a known demand for a product. Distributor demand can be satisfied from a set of four depots, or directly from a set of two factories. Each depot can support a maximum volume of product moving through it, and each factory can produce a maximum amount of product. There are known costs associated with transporting the product, from a factory to a depot, from a depot to a distributor, or from a factory directly to a distributor.

**Sets**
- Factories = {NewDelhi, Vishakhapatnam}
- Depots = {Ahmedabad, Kolkata, Coimbatore, Nagpur}
- Distributors = {C1, C2, C3, C4, C5, C6}
- Cities = Factories ∪ Depots ∪ Distributors

**Parameters**
- cost: Cost of shipping one ton from source  s  to destination  t.
- supply: Maximum possible supply from factory  f  (in tons).
- throughput: Maximum possible flow through depot  d  (in tons).
- demand: Demand for goods at distributor  c  (in tons).

**Decision Variables**
- flow: Quantity of goods (in tons) that is shipped from source  s  to destionation  t.

**Objective Function**
- Cost: Minimize total shipping costs.
**Minimize = cost ∗ flow**
 
**Constraints**
Factory output: Flow of goods from a factory must respect maximum capacity.
- flow ≤ supply
 
Distributor demand: Flow of goods must meet distributor demand.
- flow = demand
 
Depot flow: Flow into a depot equals flow out of the depot.
- flow = flow (from source to depot and depot to destination)
 
- Depot capacity: Flow into a depot must respect depot capacity.
flow ≤ throughput

In [18]:
# Create dictionaries to capture factory supply limits, depot throughput limits, and distributor demand.
supply = dict({'NewDelhi': 150000,
               'Vishakhapatnam': 200000})

throughput = dict({'Ahmedabad': 70000,
                   'Kolkata': 50000,
                   'Coimbatore': 100000,
                   'Nagpur': 40000})

demand = dict({'C1': 50000,
               'C2': 10000,
               'C3': 40000,
               'C4': 35000,
               'C5': 60000,
               'C6': 20000})

# Create a dictionary to capture shipping costs.
arcs, cost = gp.multidict({
    ('NewDelhi', 'Ahmedabad'): 0.5,
    ('NewDelhi', 'Kolkata'): 0.5,
    ('NewDelhi', 'Coimbatore'): 1.0,
    ('NewDelhi', 'Nagpur'): 0.2,
    ('NewDelhi', 'C1'): 1.0,
    ('NewDelhi', 'C3'): 1.5,
    ('NewDelhi', 'C4'): 2.0,
    ('NewDelhi', 'C6'): 1.0,
    ('Vishakhapatnam', 'Kolkata'): 0.3,
    ('Vishakhapatnam', 'Coimbatore'): 0.5,
    ('Vishakhapatnam', 'Nagpur'): 0.2,
    ('Vishakhapatnam', 'C1'): 2.0,
    ('Ahmedabad', 'C2'): 1.5,
    ('Ahmedabad', 'C3'): 0.5,
    ('Ahmedabad', 'C5'): 1.5,
    ('Ahmedabad', 'C6'): 1.0,
    ('Kolkata', 'C1'): 1.0,
    ('Kolkata', 'C2'): 0.5,
    ('Kolkata', 'C3'): 0.5,
    ('Kolkata', 'C4'): 1.0,
    ('Kolkata', 'C5'): 0.5,
    ('Coimbatore', 'C2'): 1.5,
    ('Coimbatore', 'C3'): 2.0,
    ('Coimbatore', 'C5'): 0.5,
    ('Coimbatore', 'C6'): 1.5,
    ('Nagpur', 'C3'): 0.2,
    ('Nagpur', 'C4'): 1.5,
    ('Nagpur', 'C5'): 0.5,
    ('Nagpur', 'C6'): 1.5
})

In [19]:
model = gp.Model('SupplyNetworkDesign')
flow = model.addVars(arcs, obj=cost, name="flow")
flow

{('NewDelhi', 'Ahmedabad'): <gurobi.Var *Awaiting Model Update*>,
 ('NewDelhi', 'Kolkata'): <gurobi.Var *Awaiting Model Update*>,
 ('NewDelhi', 'Coimbatore'): <gurobi.Var *Awaiting Model Update*>,
 ('NewDelhi', 'Nagpur'): <gurobi.Var *Awaiting Model Update*>,
 ('NewDelhi', 'C1'): <gurobi.Var *Awaiting Model Update*>,
 ('NewDelhi', 'C3'): <gurobi.Var *Awaiting Model Update*>,
 ('NewDelhi', 'C4'): <gurobi.Var *Awaiting Model Update*>,
 ('NewDelhi', 'C6'): <gurobi.Var *Awaiting Model Update*>,
 ('Vishakhapatnam', 'Kolkata'): <gurobi.Var *Awaiting Model Update*>,
 ('Vishakhapatnam', 'Coimbatore'): <gurobi.Var *Awaiting Model Update*>,
 ('Vishakhapatnam', 'Nagpur'): <gurobi.Var *Awaiting Model Update*>,
 ('Vishakhapatnam', 'C1'): <gurobi.Var *Awaiting Model Update*>,
 ('Ahmedabad', 'C2'): <gurobi.Var *Awaiting Model Update*>,
 ('Ahmedabad', 'C3'): <gurobi.Var *Awaiting Model Update*>,
 ('Ahmedabad', 'C5'): <gurobi.Var *Awaiting Model Update*>,
 ('Ahmedabad', 'C6'): <gurobi.Var *Awaiting Mod

In [7]:
# Production capacity limits
factories = supply.keys()
factory_flow = model.addConstrs((gp.quicksum(flow.select(factory, '*')) <= supply[factory]
                                 for factory in factories), name="factory")
factory_flow

{'Liverpool': <gurobi.Constr *Awaiting Model Update*>,
 'Brighton': <gurobi.Constr *Awaiting Model Update*>}

In [13]:
# Distributor demand
distributors = demand.keys()
distributor_flow = model.addConstrs((gp.quicksum(flow.select('*', distributor)) == demand[distributor]
                                  for distributor in distributors), name="distributor")

In [14]:
# Depot flow conservation
depots = through.keys()
depot_flow = model.addConstrs((gp.quicksum(flow.select(depot, '*')) == gp.quicksum(flow.select('*', depot))
                               for depot in depots), name="depot")

In [15]:
# Depot throughput
depot_capacity = model.addConstrs((gp.quicksum(flow.select('*', depot)) <= through[depot]
                                   for depot in depots), name="depot_capacity")

In [16]:
model.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 16 rows, 29 columns and 65 nonzeros
Model fingerprint: 0x3607c855
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 2e+05]
Presolve removed 1 rows and 0 columns
Presolve time: 0.02s
Presolved: 15 rows, 29 columns, 64 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4800000e+05   1.812500e+04   0.000000e+00      0s
       7    1.9850000e+05   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.03 seconds
Optimal objective  1.985000000e+05


In [17]:
product_flow = pd.DataFrame(columns=["From", "To", "Flow"])
for arc in arcs:
    if flow[arc].x > 1e-6:
        product_flow = product_flow.append({"From": arc[0], "To": arc[1], "Flow": flow[arc].x}, ignore_index=True)  
product_flow.index=[''] * len(product_flow)
product_flow

,From,To,Flow
,Liverpool,C1,50000.0
,Liverpool,C6,20000.0
,Brighton,Birmingham,50000.0
,Brighton,London,55000.0
,Brighton,Exeter,40000.0
,Birmingham,C2,10000.0
,Birmingham,C4,35000.0
,Birmingham,C5,5000.0
,London,C5,55000.0
,Exeter,C3,40000.0
